In [244]:
import os
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skimage.feature import graycomatrix, graycoprops
import cv2
import skimage as si
from scipy import ndimage, spatial
from detecto import core
from scipy import ndimage as ndi



In [292]:
# Set working directory and input dataset
dataset = 'S1-2006'
tray = 'Tray1'
thr = 0.8



In [294]:
#determine features using distance map and local maxima

F = np.empty([2,150])

for i in range(1, 11):
    for j in range(1, 16): 
        
        n = ((i-1)*15-1) + (j)        
        
        pod = plt.imread(os.getcwd()+'\\Data\\'+dataset+'\\Podimages\\'+tray+'\\'+dataset+'_'+tray+'_'+str(i)+'.'+str(j)+'.png')[:,:,:3]
        pod = cv2.resize(pod, (260, 260), interpolation = cv2.INTER_AREA)
        podgray = cv2.cvtColor(pod, cv2.COLOR_BGR2GRAY)
        thr_gray = cv2.threshold((podgray*255).astype(np.uint8), 10, 255, cv2.THRESH_BINARY)[1]
        mask = ndimage.binary_fill_holes(thr_gray).astype(np.uint8)

        podsat = (np.round(255*mask*cv2.cvtColor(pod, cv2.COLOR_RGB2HSV)[:,:,1])).astype(np.uint8)
        ret, thresh = cv2.threshold(podsat, 200, 255, cv2.THRESH_TOZERO_INV)
        ret, thresh = cv2.threshold(thresh, 70, 255, cv2.THRESH_BINARY)
        podbin = si.morphology.binary_opening(ndimage.binary_closing(thresh, np.ones((6,6))), np.ones((20,20))).astype(np.uint8)
        
        contours, hierarchy = cv2.findContours(podbin,2,1)
        
        if  len(contours) == 0:
            F[:,n] = np.array([0, 0])
   
        else:    
            F[0,n] = np.sum(podbin)/1000 #area devided by 100 for scaling
    
            mass_x, mass_y = np.where(podbin == 1)
            c = np.empty([1,2])
            c[0,:] = np.array([np.average(mass_x), np.average(mass_y)])

            model = core.Model.load('model_weights_2.pth', ['Basil', 'Nothing'])
            labels, boxes, scores = model.predict(pod)
            ind = np.argwhere(scores>thr)
            box = boxes[ind, :][0]
            coor = np.empty([box.shape[0], 2]).astype(np.uint8)
                                
            for j in range(box.shape[0]):
                coor[j,0] = int((box[j,0]+box[j,2])/2)
                coor[j,1] = int((box[j,1]+box[j,3])/2)
                 
            F[1, n] = (coor.shape[0]-len(contours))
            
np.save(os.getcwd()+'\\Data\\'+dataset+'\\Model\\features_'+tray+'.npy', F)


In [ ]:
#optional version containing all tested features

F = np.empty([13,150])

for i in range(1, 11):
    for j in range(1, 16): 
        
        n = ((i-1)*15-1) + (j)        
        
        pod = plt.imread(os.getcwd()+'\\Data\\'+dataset+'\\Podimages\\'+tray+'\\'+dataset+'_'+tray+'_'+str(i)+'.'+str(j)+'.png')[:,:,:3]
        pod = cv2.resize(pod, (260, 260), interpolation = cv2.INTER_AREA)
        podgray = cv2.cvtColor(pod, cv2.COLOR_BGR2GRAY)
        thr_gray = cv2.threshold((podgray*255).astype(np.uint8), 10, 255, cv2.THRESH_BINARY)[1]
        mask = ndimage.binary_fill_holes(thr_gray).astype(np.uint8)

        podsat = (np.round(255*mask*cv2.cvtColor(pod, cv2.COLOR_RGB2HSV)[:,:,1])).astype(np.uint8)
        ret, thresh = cv2.threshold(podsat, 200, 255, cv2.THRESH_TOZERO_INV)
        ret, thresh = cv2.threshold(thresh, 70, 255, cv2.THRESH_BINARY)
        podbin = si.morphology.binary_opening(ndimage.binary_closing(thresh, np.ones((6,6))), np.ones((20,20))).astype(np.uint8)
        
        contours, hierarchy = cv2.findContours(podbin,2,1)
        
        if  len(contours) == 0:
            F[:5,n] = np.array([0, 0, 0, 0, 0])
   
        else:    
            F[0,n] = np.sum(podbin)/100 #area devided by 100 for scaling
    
            mass_x, mass_y = np.where(podbin == 1)
            c = np.empty([1,2])
            c[0,:] = np.array([np.average(mass_x), np.average(mass_y)])

            model = core.Model.load('model_weights_2.pth', ['Basil', 'Nothing'])
            labels, boxes, scores = model.predict(pod)
            ind = np.argwhere(scores>thr)
            box = boxes[ind, :][0]
            coor = np.empty([box.shape[0], 2]).astype(np.uint8)
                                
            for j in range(box.shape[0]):
                coor[j,0] = int((box[j,0]+box[j,2])/2)
                coor[j,1] = int((box[j,1]+box[j,3])/2)
                 
            F[1, n] = len(contours)*10
            F[2, n] = coor.shape[0]*10
            
            if (coor.shape[0] == 0 or coor.shape[0] == 1):
                F[1:5, n] = np.array([0, 0, 0, 0])
            else:
                dc = spatial.distance.cdist(coor, c, 'euclidean')
                F[3, n] = dc[np.nonzero(dc)].mean() # average distance of local maxima to center of mass
    
                dinter = spatial.distance.cdist(coor, coor, 'euclidean')
                F[4, n] = dinter[np.nonzero(dinter)].mean() # average distance between local maxima
            
        glcm1 = graycomatrix(podsat, [1], [0], levels=256, symmetric=False, normed=True)
        F[5,n] = graycoprops(glcm1, 'contrast')[0][0]/2
        F[6,n] = graycoprops(glcm1, 'energy')[0][0]*100
        F[7,n] = graycoprops(glcm1, 'homogeneity')[0][0]*100
        F[8,n] = graycoprops(glcm1, 'correlation')[0][0]*100
        
        glcm2 = graycomatrix(podsat, [1], [np.pi/2], levels=256, symmetric=False, normed=True)
        F[9,n] = graycoprops(glcm2, 'contrast')[0][0]/2
        F[10,n] = graycoprops(glcm2, 'energy')[0][0]*100
        F[11,n] = graycoprops(glcm2, 'homogeneity')[0][0]*100
        F[12,n] = graycoprops(glcm2, 'correlation')[0][0]*100

np.save(os.getcwd()+'\\Data\\'+dataset+'\\Model\\features_'+tray+'.npy', F)
